<a href="https://colab.research.google.com/github/marcinbartosiewicz/ManagerSurvey/blob/main/MLModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split,RandomizedSearchCV,\
cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
import xgboost as xgb
import pickle

In [ ]:
df=pd.read_csv('/content/cleanedaskamanager.csv')

In [ ]:
dataformodel=pd.get_dummies(df)

In [ ]:
Y=dataformodel.pop('Annual_salary')

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(dataformodel,Y,random_state=23)

In [ ]:
models=[RandomForestRegressor(),ElasticNet(),xgb.XGBRegressor()]
for model in models:
  
  scores=cross_val_score(model, X_train,y_train, cv=5,
                         scoring='neg_root_mean_squared_error')
  print(-scores.mean())

31235.76197150273
37152.24224292919
29518.218122221228


In [ ]:
params = {
    
    'max_depth':[3,6,10],
    'n_estimators': [50,100,200],
    'learning_rate':[0.01,0.1,0.3],
    'subsample': [0.1,0.5,1],
    'colsample_bytree': [0.1,0.5,1],
}
model=xgb.XGBRegressor()
random=RandomizedSearchCV(model,params,scoring='neg_root_mean_squared_error'
,n_jobs=-1,n_iter=20)
random.fit(X_train,y_train)

RandomizedSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None,...
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n_jobs=None,
                                          num_parallel_tree=None,
                                          predictor=None, random_state=None, ...),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.1, 0.5, 1],
                                        'learning_rate': [0.01, 0.1, 0.3],
                                        'max_depth': [3, 6, 10],
                                        'n_estimators': [50, 100, 200],
                                        'subsample': [0.1, 0.5, 1]},
                   scoring='neg_root_mean_squared_error')

In [ ]:
random.best_params_

{'subsample': 1,
 'n_estimators': 200,
 'max_depth': 3,
 'learning_rate': 0.3,
 'colsample_bytree': 1}

In [ ]:
model=random.best_estimator_
scores=cross_val_score(model, X_train,y_train, cv=5
,scoring='neg_root_mean_squared_error')
-scores.mean()

28987.378312460904

In [ ]:
y_pred=model.predict(X_test)
mean_squared_error(y_test,y_pred,squared=False)

29113.59662851414

In [ ]:
scores=cross_val_score(model, dataformodel,Y, cv=5
,scoring='neg_root_mean_squared_error')
-scores.mean()

29875.15133374549

In [ ]:
model.fit(dataformodel,Y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [ ]:
filename = 'MLModel.sav'
pickle.dump(model, open(filename, 'wb'))